In [ ]:
import os, gc
import sys, pdb
import copy, time
import json, random

import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from pathlib import Path

import matplotlib
import os

import torch

from matplotlib import colors
import matplotlib.pyplot as plt
from colorama import Style, Fore
# %matplotlib inline ipnyb

from dataloader.my_loader import MyDataLoader
from dataloader.OAutoEncoder_loader import ARCDataset

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from visualization.my_vis import ARCPlottor


from models.O_Auto_Encoder import *

for dirname, _, filenames in os.walk('/home/hyunseok/enssamble/settings'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

local_path = "/home/hyunseok/enssamble/data/kaggle/"
with open("/home/hyunseok/enssamble/settings/kaggle_data_file_name.json",'r') as file:
    path_dict = json.load(file)
    data = MyDataLoader("arcprize", path_dict, local_path) 
    data.cur_data_mode("train")

plotter = ARCPlottor()


cmap = colors.ListedColormap(
    ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
     '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])

norm = colors.Normalize(vmin=0, vmax=9)
color_list = ["black", "blue", "red", "green", "yellow", "gray", "magenta", "orange", "sky", "brown"]


In [ ]:
!g++ -pthread -lpthread -O3 -std=c++17 -o ./cookedModels/SkleanTree ./models/SkleanTree.cpp
!./cookedModels/SkleanTree

In [3]:
BASE_FOLDER = '/home/hyunseok/enssamble/data/kaggle/'
file_list = [
    f'{BASE_FOLDER}/arc-agi_training_challenges.json',
    f'{BASE_FOLDER}/arc-agi_evaluation_challenges.json',
    f'{BASE_FOLDER}/arc-agi_test_challenges.json'

]
def auto_encoder():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
        '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])

    norm = colors.Normalize(vmin=0, vmax=10)

    train_dataset = ARCDataset(file_list)
    train_dataset.set_dataset()
    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

    Dimension = train_dataset.dim
    Keys = train_dataset.keys

    IN_DIM = 100 + 1
    OUT_DIM = 100
    LATENT_DIM = 1024
    model = LSTM(IN_DIM, OUT_DIM, LATENT_DIM).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    training = Training(model, train_loader, criterion, optimizer, device)
    training.train()

    model.load_state_dict(torch.load('model.pth'))

    sanity = Prediction(model, train_dataset, Dimension, cmap, norm, device)
    def plot_pic(x):
        plt.imshow(np.array(x), cmap=cmap, norm=norm)
        plt.show()
    for i in range(100):
        result = sanity.predict(model,[sanity.data.data[i][1],sanity.data.data[i][2]])
        print(result)
        output1 = sanity.remove_tail_zeros(sanity.data.data[i][0], torch.round(result * 9), 'out')
        plot_pic(output1)
# for i in range(len(pred.data)):
#     result = pred.predict(model,[pred.data.data[i][1],pred.data.data[i][2]])
#     output1 = pred.remove_tail_zeros(pred.data.data[i][0], torch.round(result * 9), 'out')
#     plot_pic(output1)

In [4]:
from models.symmetry_repariring import *
from models.color_counter import *
from models.different_solvers import *
from models.via_tree import *
from cookedModels.utils import *
from visualization.my_vis import *

from checker.checker import depency_inject_funcs

In [6]:

import json
from datetime import datetime

def make_submission_file(data, solver_name, dataloader, data_mode):
    now = datetime.now()
 

    metadata = {
        "date": now.strftime("%Y-%m-%d"),  # YYYY-MM-DD 형식
        "time": now.strftime("%H:%M:%S"),   # HH:MM:SS 형식
        "data_mode": data_mode,
        "solver_name": solver_name,
    } 
    json_file_data = {
        'metadata': metadata,
        'submission': data
    }

    # JSON 파일로 저장
    with open('./data/submissions/{}-{}-{}-{}.json'.format(metadata['date'], metadata['time'],metadata['data_mode'], metadata['solver_name'] ), 'w') as json_file:
        json.dump(json_file_data, json_file, indent=4)

In [7]:



data_mode = "evaluation"

f1 = lambda test_input: predict_repeating(test_input)
f2 = lambda task, test_input: predict_transforms_grid_2x(task, test_input)
f3 = lambda x: predict_chess(grid_filter(x))
f4 = lambda task, test_input: predict_tiles_shape(task, test_input)
f5 = lambda task,test_input: predict_grid_transforms(task, test_input)
f6 = lambda test_input: predict_repeating_mask(test_input)

fdata1 = depency_inject_funcs( data, data_mode, f1, [(check_repeating, [True], {})])
fdata2 = depency_inject_funcs( data, data_mode, f2, [(check_grid, [], {}),(check_sub_grid_2x,[],{})])
fdata3 = depency_inject_funcs( data, data_mode, f3, [(check_grid, [], {}),(check_chess,[ False, True],{})])
fdata4 = depency_inject_funcs( data, data_mode, f4, [(check_tiles_shape, [True], {})],[],{})
fdata5 = depency_inject_funcs( data, data_mode, f5, [(check_grid, [], {}), (check_grid_transforms,[],{})])
fdata6 = depency_inject_funcs( data, data_mode, f6, [(check_sub_mask, [], {})])



In [ ]:
from cookedModels.colors_counter import *
from cookedModels.symmetry_repairing import *
from cookedModels.sklearn_tree import * 

store1 = {}
store2 = {}
store3 = {}
for task_id in data.cur_problem:
    task = data.cur_problem[task_id]
    for i in range(len(task['test'])):
        test_input = np.array(task['test'][i]['input'])
        prn = []
        ganswer = []
        cooked_symmetry_repairing(store1, task, task_id, i, ganswer, prn, pic_mode=True)
        prn = []
        ganswer = []
        cooked_color_counter(store2, task, task_id, i, ganswer, prn, pic_mode=True)
        prn = []
        ganswer = []
        cooked_sklearn_tree(store3, task, test_input, task_id, prn, pic_mode=True)


In [ ]:


make_submission_file(fdata1, "predict_repeating", data, data_mode)
make_submission_file(fdata2, "predict_transforms_grid_2x", data, data_mode)
make_submission_file(fdata3, "predict_chess", data, data_mode)
make_submission_file(fdata4, "predict_tiles_shape", data, data_mode)
make_submission_file(fdata5, "predict_grid_transforms", data, data_mode)
make_submission_file(fdata6, "predict_repeating_mask", data, data_mode)

make_submission_file(store1, "symmetry_repairing", data, data_mode)
make_submission_file(store2, "colors_counter", data, data_mode)
make_submission_file(store3, "sklearn_tree", data, data_mode)